In [57]:
from importlib import reload
reload(change)
reload(base)

TypeError: reload() argument must be a module

In [1]:
import pandas as pd
import numpy as np
import src.in_out as io
import src.info as info
import src.condition as con
import src.base as base
from itertools import combinations
from src.spread import get_spread
import src.change as change

## Veri Hazırlama

In [2]:
#Verilere ait path'ler alınıyor
path_list=io.get_path('data')

In [3]:
#Okunacak sütun adları
cols= ["Time","BID price","ASK price"]
#Tüm veriler okunuyor
master_data=io.read_allData(path_list, cols=cols)
master_data.columns=['time','bid_price','ask_price','name','date']

In [4]:
#date sütunu Timestamp'e dönüştürülüyor ve tarih ve saat bilgisi birleştiriliyor
master_data['date']=pd.to_datetime(master_data['date'], errors='coerce')
master_data['date'] += pd.to_timedelta(master_data.pop('time').astype(str))

In [5]:
#zaman dilimi ekleniyor
master_data['time_period']=master_data.date.dt.hour

## Mid Price

In [6]:
# mid price hesaplanıyor
master_data['mid_price']=base.average_of_series(master_data.bid_price,master_data.ask_price)

In [7]:
# spread hesaplama için master_data, uygun bir forma dönüştürülüyor
df_mid_price=master_data.pivot(index='date', columns='name', values='mid_price')

## Spread
<img src="img/spread_flowchart.png">
<br/>
Spread, her bir saat için ayrı ayrı hesaplanmıştır

In [8]:
values={'a_PNLTICK':1,
'a_TICKSIZE':1,
'b_PNLTICK':1,
'b_TICKSIZE':1}

In [9]:
# pair'lar alınıyor
pairs=combinations(df_mid_price.columns, 2)

In [10]:
#spread hesaplanıyor
spread_list=[
    base.groupby_date_time(df_mid_price.loc[:,pair]).apply(lambda x: get_spread(x,values))\
        for pair in pairs]
df_spread=pd.concat(spread_list,axis=1).droplevel([0,1])

## Change

In [11]:
#change alınıyor
df_change=base.groupby_date_time(df_spread).apply(change.get_change)
df_change=df_change.droplevel([0,1])

## Amplitude

In [12]:
all_pairs_of_amplitude=[]

In [13]:
for pair_change in df_change:   #Her bir pair alınıyor
    pair_group = list(base.groupby_date_time(df_change[pair_change])) #gün ve saate göre gruplanıyor
    sub_amplitudes=[]
    for i in range(len(pair_group)):
        sub_change=base.to_series(pair_group[i][1])
        sub_amplitudes.append(change.get_amplitude(sub_change)) #pair'a ait her alt grubun amplitudu'u hesaplanıyor
    all_pairs_of_amplitude.append(pd.concat(sub_amplitudes)) # alt amplitude'lar birleştiriliyor

#### 1. Amplitude'u hesaplanan tüm pair'lar all_pairs_of_amplitude adlı dizide tutuluyor.  Hepsi tek DataFrame'de birleştirilmek istenirse aşağıdaki kod çalıştırılabilir.

In [14]:
from functools import reduce

df_final = reduce(lambda left,right: pd.merge(left,right,on=['date','duration'],how='outer'), all_pairs_of_amplitude)

df_final # NaN'lar birleştirmeden dolayı. ÖNEMSİZ ! 6AU8_6CU8, 2018-09-06 00:00:49 dan başladığından öncesi NaN

,date,duration,6AU8_6BU8,6AU8_6CU8,6BU8_6CU8
0,2018-09-06 00:00:17,00:00:13,-0.000218,NaN,0.000218
1,2018-09-06 00:00:39,00:00:22,0.000050,NaN,-0.000050
2,2018-09-06 00:00:48,00:00:09,-0.000059,NaN,0.000059
3,2018-09-06 00:00:49,00:00:01,0.000047,-0.000003,-0.000050
4,2018-09-06 00:00:50,00:00:01,-0.000047,0.000003,0.000050
...,...,...,...,...,...
11864,2018-09-07 23:51:36,00:00:05,NaN,NaN,0.000050
11865,2018-09-07 23:55:36,00:00:01,NaN,NaN,0.000050
11866,2018-09-07 23:55:54,00:00:17,NaN,NaN,0.000100
11867,2018-09-07 23:58:52,00:00:02,NaN,NaN,-0.000100


#### 2. Henüz condition uygulanmamış sonuçlar excel'e yazdırabilir.

In [15]:
df_mid_price.to_excel('out/mid_price.xlsx',)
df_spread.to_excel('out/spread.xlsx')
df_change.to_excel('out/change.xlsx')
df_final.to_excel('out/amplitude.xlsx')

# Koşula Göre Amplitude ve Duration Verisinin Yeniden Düzenlenmesi

In [16]:
#Önce condition modülümüzü import ediyoruz.
import src.condition as condition

In [17]:
# Hesaplayacağımız verileri saklayabilmek için boş bir dizi yarattık
condition_data = []

In [18]:
# Python'a ait lambda ifadelerini kullanabilmek için basit bir fonksiyon yazdık
def uygula(data):
    """parametre olarak gelen 'data' daki indeksi kaldırır.
       percentile(.6) ya göre koşul uygulandı.
       percentile silinip yerine medyan() yazılırsa, koşulu medyana göre hesaplar.
       ya da percentile'ın '.6' olan parametresi değiştirilebilir.
    """
    data = data.reset_index(drop=True)
    signs = condition.Sign(data).percentile(.6) #
    return condition.apply(data,signs)

In [19]:
# bir for döngüsü ile tüm pairlara erişiyoruz
for data in all_pairs_of_amplitude:
    #groupby içerisindeki floor metoduyla veriyi neye göre gruplayacağımızı belirtiyoruz.
    #Burada groupby 2 parametre aldı. ilk parametre ile 'güne göre', 2. parametre ile de 'saate' göre
    #ayırıp, 'uygula' metodunu her gruba ayrı ayrı uygulayacağını belirttik
    """ 'd' -> day
        '2h' -> 2şer saat
        'h'-> birer saat
        'nd' ->n güne böl
        'nh' -> n saate böl gibi        
    """ 
    """.apply metodu ile yapmak istediğimiz işlemi belirtiyoruz
        lambda x : uygula(x) -> her bir grubu sırayla
        x olarak adlandır ve uygula fonksiyonuna parametre olarak gönder
    """
    result = data.groupby([data.date.dt.floor('d'),data.date.dt.floor('h')])\
        .apply(lambda x : uygula(x)).droplevel([0,1])
    #son olarak hesaplanan verileri condition_data adlı listeye atmasını söylüyoruz
    condition_data.append(result)

duration= {'pozitif': 12.0, 'negatif': nan}
amplitude= {'pozitif': 9.649999999994385e-05, 'negatif': -5.8999999999809205e-05}
PAIR:  6AU8_6BU8
duration= {'pozitif': 6.0, 'negatif': nan}
amplitude= {'pozitif': 9.649999999994385e-05, 'negatif': -5.8999999999809205e-05}
PAIR:  6AU8_6BU8
duration= {'pozitif': 6.0, 'negatif': nan}
amplitude= {'pozitif': 9.650000000005488e-05, 'negatif': -8.969999999999295e-05}
PAIR:  6AU8_6BU8
duration= {'pozitif': 5.0, 'negatif': nan}
amplitude= {'pozitif': 0.00010899999999969268, 'negatif': -9.649999999994385e-05}
PAIR:  6AU8_6BU8
duration= {'pozitif': 4.0, 'negatif': nan}
amplitude= {'pozitif': 0.00010899999999991472, 'negatif': -5.900000000014227e-05}
PAIR:  6AU8_6BU8
duration= {'pozitif': 9.0, 'negatif': nan}
amplitude= {'pozitif': 5.900000000003125e-05, 'negatif': -5.900000000003125e-05}
PAIR:  6AU8_6BU8
duration= {'pozitif': 14.0, 'negatif': nan}
amplitude= {'pozitif': 9.650000000005488e-05, 'negatif': -5.8999999999809205e-05}
PAIR:  6AU8_6BU8
durati

duration= {'pozitif': 6.0, 'negatif': nan}
amplitude= {'pozitif': 9.999999999998899e-05, 'negatif': -5.0000000000105516e-05}
PAIR:  6BU8_6CU8
duration= {'pozitif': 5.0, 'negatif': nan}
amplitude= {'pozitif': 9.999999999998899e-05, 'negatif': -5.0000000000105516e-05}
PAIR:  6BU8_6CU8
duration= {'pozitif': 4.0, 'negatif': nan}
amplitude= {'pozitif': 9.999999999998899e-05, 'negatif': -9.999999999976694e-05}
PAIR:  6BU8_6CU8
duration= {'pozitif': 9.0, 'negatif': nan}
amplitude= {'pozitif': 5.0000000000105516e-05, 'negatif': -5.0000000000105516e-05}
PAIR:  6BU8_6CU8
duration= {'pozitif': 14.0, 'negatif': nan}
amplitude= {'pozitif': 9.999999999998899e-05, 'negatif': -5.0000000000105516e-05}
PAIR:  6BU8_6CU8
duration= {'pozitif': 5.0, 'negatif': nan}
amplitude= {'pozitif': 9.999999999998899e-05, 'negatif': -5.0000000000105516e-05}
PAIR:  6BU8_6CU8
duration= {'pozitif': 5.0, 'negatif': nan}
amplitude= {'pozitif': 9.999999999998899e-05, 'negatif': -5.0000000000105516e-05}
PAIR:  6BU8_6CU8
durat

In [20]:
# 1. pairi görüntüleyelim

In [43]:
condition_data[0]

,date,duration,6AU8_6BU8
0,2018-09-06 00:00:17,00:00:13,-0.000218
1,2018-09-06 00:00:39,00:00:22,0.000050
2,2018-09-06 00:01:22,00:00:43,-0.000172
3,2018-09-06 00:01:24,00:00:02,0.000109
4,2018-09-06 00:02:11,00:00:47,-0.000059
...,...,...,...
193,2018-09-07 23:56:50,00:00:16,-0.000062
194,2018-09-07 23:57:25,00:00:35,0.000062
195,2018-09-07 23:58:50,00:01:25,-0.000200
196,2018-09-07 23:58:54,00:00:04,0.000111


## Aralık Oluşturma

Pair'lerden biri olan **6AU8_6BU8**'deki verileri aralığa yerleştirelim. Bunun için **interval** modülündeki **set_range** ve **set_timeRange** fonksiyonlarını kullanacağız.
<span style="text-decoration:underline">amplitude</span> için **set_range**, <span style="text-decoration:underline">duration</span> için ise **set_timeRange** kullanacağız. Duration'nın veri tipi **timedelta64** olduğundan ayrı bir metot yazıldı.<br/>
Fonksiyonlar **data**, **value**, ve **method** olmak üzere 3 parametre alarak çalışır. Parametreler hakkında kısa bilgi:
1. **data** : aralığa yerleştirilecek veriler. örn. duration veya amplitude
2. **method** : aralık oluşturma yöntemini burada belirtiyoruz. **2** çeşit aralık oluşturma yöntemi var. **'width'** ve **'size'**
   *  **width** : yöntemi seçilirse her bir aralığın genişliği **value** değeri kadardır.
   *  **size** : yöntemi seçilirse **@value** adet aralık oluşturur. Her aralığın genişliği aynıdır.
   *  **value** : yöntem 'width' ise aralık genişliğini, 'size' ise aralık adetini belirtir.

**6AU8_6BU8** pair'ini alalım. Duration'ı **5'er sn'lik aralıklara** yerleştirelim. Amplitude verisini ise **aralık sayısı 100 olacak** ve **her aralığın genişliği eşit** olacak şekilde aralıklara yerleştirelim. Böylece her iki yöntemi de görmüş olacağız

In [22]:
# 6AU8_6BU8, condition_data adlı dizinin 0. elemanı
ab = condition_data[0]
ab

,date,duration,6AU8_6BU8
0,2018-09-06 00:00:17,00:00:13,-0.000218
1,2018-09-06 00:00:39,00:00:22,0.000050
2,2018-09-06 00:01:22,00:00:43,-0.000172
3,2018-09-06 00:01:24,00:00:02,0.000109
4,2018-09-06 00:02:11,00:00:47,-0.000059
...,...,...,...
193,2018-09-07 23:56:50,00:00:16,-0.000062
194,2018-09-07 23:57:25,00:00:35,0.000062
195,2018-09-07 23:58:50,00:01:25,-0.000200
196,2018-09-07 23:58:54,00:00:04,0.000111


In [23]:
#6AU8_6BU8 'den duration ve amplitude sütunları alınıyor
duration = ab['duration']
amplitude = ab['6AU8_6BU8']

In [24]:
# interval modülünden, kullanacağımız fonksiyonları import ediyoruz
from src.interval import set_range, set_timeRange

In [25]:
# duration için her biri 5'er sn'lik aralık oluşturuyoruz.
new_duration=set_timeRange(data=duration, value=5, method='width')
new_duration

0      (0 days 00:00:10, 0 days 00:00:15]
1      (0 days 00:00:20, 0 days 00:00:25]
2      (0 days 00:00:40, 0 days 00:00:45]
3      (0 days 00:00:00, 0 days 00:00:05]
4      (0 days 00:00:45, 0 days 00:00:50]
                      ...                
193    (0 days 00:00:15, 0 days 00:00:20]
194    (0 days 00:00:30, 0 days 00:00:35]
195    (0 days 00:01:20, 0 days 00:01:25]
196    (0 days 00:00:00, 0 days 00:00:05]
197    (0 days 00:00:05, 0 days 00:00:10]
Name: duration, Length: 4355, dtype: category
Categories (88, interval[timedelta64[ns]]): [(0 days 00:00:00, 0 days 00:00:05] < (0 days 00:00:05, 0 days 00:00:10] < (0 days 00:00:10, 0 days 00:00:15] < (0 days 00:00:15, 0 days 00:00:20] ... (0 days 00:07:00, 0 days 00:07:05] < (0 days 00:07:05, 0 days 00:07:10] < (0 days 00:07:10, 0 days 00:07:15] < (0 days 00:07:15, 0 days 00:07:20]]

In [26]:
# amplitude için her biri eşit büyüklükte 100 adet aralık oluşturuldu ve veriler aralıklara yerleştirildi
new_amplitude = set_range(data=amplitude, value=100, method='size')
new_amplitude

0      (-0.000358, 0.019]
1      (-0.000358, 0.019]
2      (-0.000358, 0.019]
3      (-0.000358, 0.019]
4      (-0.000358, 0.019]
              ...        
193    (-0.000358, 0.019]
194    (-0.000358, 0.019]
195    (-0.000358, 0.019]
196    (-0.000358, 0.019]
197    (-0.000358, 0.019]
Name: 6AU8_6BU8, Length: 4355, dtype: category
Categories (99, interval[float64]): [(-0.91, -0.89] < (-0.89, -0.871] < (-0.871, -0.852] < (-0.852, -0.832] ... (0.928, 0.948] < (0.948, 0.967] < (0.967, 0.986] < (0.986, 1.006]]

In [28]:
#pandas'ın concat metodunu kullarak hesapladığımız aralıkları DataFrame'de gösterdik
pd.concat([new_duration,new_amplitude],axis=1)

,duration,6AU8_6BU8
0,"(0 days 00:00:10, 0 days 00:00:15]","(-0.000358, 0.019]"
1,"(0 days 00:00:20, 0 days 00:00:25]","(-0.000358, 0.019]"
2,"(0 days 00:00:40, 0 days 00:00:45]","(-0.000358, 0.019]"
3,"(0 days 00:00:00, 0 days 00:00:05]","(-0.000358, 0.019]"
4,"(0 days 00:00:45, 0 days 00:00:50]","(-0.000358, 0.019]"
...,...,...
193,"(0 days 00:00:15, 0 days 00:00:20]","(-0.000358, 0.019]"
194,"(0 days 00:00:30, 0 days 00:00:35]","(-0.000358, 0.019]"
195,"(0 days 00:01:20, 0 days 00:01:25]","(-0.000358, 0.019]"
196,"(0 days 00:00:00, 0 days 00:00:05]","(-0.000358, 0.019]"


Yukarıda görüldüğü gibi verilerimizi aralığa yerleştirdik. Fakat her biri Interval türünden olduğundan dolayı hareket alanımızı kısıtlayacaktır.
Örneğin aşağıdaki gibi sütun verileriyle bir toplama işlemi gerçekleştirmek istediğimizi varsayalım :
```python
new_amplitude.sum()
```
Bu hamlemiz aşağıdaki hata ile sonuçlanacktır:
```
TypeError: Categorical cannot perform the operation sum
```
Aralık tanımlamadaki amaç, bu verileri **kategorilendirmek** olduğundan, her bir aralığın yalnızca **sağ değeri** ya da **ortalamasıyla** çalışmamız işlemlerimizin sonucunu etkilemeyecektir. Bize istatistiksel hesaplamalar veya grafik işlemlerinde kolaylık sağlayacaktır.<br/>

Şimdi duration ve amplitude aralıklarının ortalamasını aşağıdaki gibi alalım. Önce **interval** modülümüzden **get_mid** fonksiyonumuzu import edelim.

In [49]:
from src.interval import get_mid

In [73]:
#Amplitude için aralıkların ortalamasını al ve veriyi float'a dönüştür
new_amplitude=get_mid(new_amplitude).astype('float64')

In [72]:
#Duration için aralıkların ortalamasını al ve veriyi timedelta64'e dönüştür
new_duration=get_mid(new_duration).astype('timedelta64')

In [77]:
#Verileri  DataFrame ile gösterelim ve 'new_ab' adlı değişkende saklayalım
new_ab=pd.concat([new_duration,new_amplitude],axis=1)
new_ab

,duration,6AU8_6BU8
0,00:00:12.500000,0.009321
1,00:00:22.500000,0.009321
2,00:00:42.500000,0.009321
3,00:00:02.500000,0.009321
4,00:00:47.500000,0.009321
...,...,...
193,00:00:17.500000,0.009321
194,00:00:32.500000,0.009321
195,00:01:22.500000,0.009321
196,00:00:02.500000,0.009321


Aralık işlemleri bu kadar. Şimdi oluşturduğumuz **new_ab** ile **Density İşlemleri**'ni gerçekleştireceğiz.

## Density

In [96]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [139]:
%autoreload

### 1. Joint Density
Bu bölümde aşağıdaki işlemler yapılmaktadır:
1. **get_frequency:** Her bir  {duration,amplitude} değerinden kaç tane olduğunu bulma işlemidir. Örneğin duration'ı 5 amplitude'u 10 olan 133 veri var, duration:5 amplitude:7 olan 23 veri var. **Parametre olarak** yalnızca **duration** ve **amplitude** içeren bir **DataFrame** alır. 
2. **joint_density:** Her bir frekansın, toplam frekansa bölünerek tabloda ilgili yere yerleştirilmiş halidir.Daha sonra bu tablonun yatay ve dikey toplamlarını alarak **marginal density** elde edilecektir. **Parametre olarak** yalnızca **duration** ve **amplitude** içeren bir **DataFrame** alır. 

**density** modülünden kullanacağımız fonksiyonları import edelim ve işlemleri gerçekleştirelim

In [126]:
from src.density import get_frequency, joint_density

In [140]:
# frekans hesaplanıyor
get_frequency(new_ab)

,duration,6AU8_6BU8,frequency
0,00:00:02.500000,0.009321,1257
1,00:00:07.500000,0.009321,974
2,00:00:12.500000,0.009321,598
3,00:00:17.500000,0.009321,396
4,00:00:22.500000,0.009321,260
...,...,...,...
61,00:00:02.500000,0.241500,1
62,00:00:02.500000,0.202500,1
63,00:00:02.500000,0.067350,1
64,00:00:02.500000,0.048000,1


In [153]:
#joint density hesaplanıyor ve joint adlı bir değişkende saklanıyor
joint=joint_density(new_ab)
joint

6AU8_6BU8,-0.455000,-0.242500,-0.068050,-0.010029,0.009321,0.048000,0.067350,0.202500,0.241500,0.996000
duration,,,,,,,,,,
00:00:02.500000,NaN,NaN,0.000231,0.000461,0.289965,0.000231,0.000231,0.000231,0.000231,0.000231
00:00:07.500000,0.000231,0.000231,NaN,0.000461,0.224683,NaN,NaN,NaN,NaN,NaN
00:00:12.500000,NaN,NaN,NaN,0.000692,0.137947,NaN,NaN,NaN,NaN,NaN
00:00:17.500000,NaN,NaN,NaN,NaN,0.091349,NaN,NaN,NaN,NaN,NaN
00:00:22.500000,NaN,NaN,NaN,0.000923,0.059977,NaN,NaN,NaN,NaN,NaN
00:00:27.500000,NaN,NaN,NaN,0.001615,0.033218,NaN,NaN,NaN,NaN,NaN
00:00:32.500000,NaN,NaN,NaN,0.002076,0.031142,NaN,NaN,NaN,NaN,NaN
00:00:37.500000,NaN,NaN,NaN,0.000692,0.019608,NaN,NaN,NaN,NaN,NaN
00:00:42.500000,NaN,NaN,NaN,0.000461,0.019377,NaN,NaN,NaN,NaN,NaN


Şimdi bulduğumuz **joint density**'nin **yataydaki ve dikeydeki toplamını** alalım. Fakat bulacağımız toplamları, aşağıdaki **Marjinal Density** tablosundaki gibi, tekrardan joint dataframe'ine aktarmayacağız. **vertical_total** ve **horizontal_total** adlı iki değişkende tutacağız. Daha sonra ise bunları kullanarak **Conditional Density** hesaplayacağız.
```
Örnek Tablo
| xy           | 4             | 6             | 8             | yatay toplam |
|--------------|---------------|---------------|---------------|--------------|
| 1            | 0,07547169811 | 0,2830188679  | 0,1320754717  | 0,4905660377 |
| 2            | 0,05031446541 | 0,07547169811 | 0,1446540881  | 0,2704402516 |
| 3            | 0,1257861635  | 0,04402515723 | 0,06918238994 | 0,2389937107 |
| dikey toplam | 0,251572327   | 0,4025157233  | 0,3459119497  | 1            |
```
Ayrıca aşağıda bulacağımız **vertical_total** ve **horizontal_total**'in toplamı **1'e eşittir**.

In [161]:
# Dikey toplam
vertical_total = joint.agg('sum') #agg metodu pandas'a ait
vertical_total

6AU8_6BU8
-0.455000    0.000231
-0.242500    0.000231
-0.068050    0.000231
-0.010029    0.010611
 0.009321    0.987543
 0.048000    0.000231
 0.067350    0.000231
 0.202500    0.000231
 0.241500    0.000231
 0.996000    0.000231
dtype: float64

In [157]:
#Yatay toplam
horizontal_total = joint.agg('sum',axis=1)
horizontal_total

duration
00:00:02.500000    0.291811
00:00:07.500000    0.225606
00:00:12.500000    0.138639
00:00:17.500000    0.091349
00:00:22.500000    0.060900
00:00:27.500000    0.034833
00:00:32.500000    0.033218
00:00:37.500000    0.020300
00:00:42.500000    0.019839
00:00:47.500000    0.013149
00:00:52.500000    0.011995
00:00:57.500000    0.008766
00:01:02.500000    0.006228
00:01:07.500000    0.005767
00:01:12.500000    0.003691
00:01:17.500000    0.003460
00:01:22.500000    0.006690
00:01:27.500000    0.003230
00:01:32.500000    0.002768
00:01:37.500000    0.001384
00:01:42.500000    0.001384
00:01:47.500000    0.001845
00:01:52.500000    0.000923
00:01:57.500000    0.000692
00:02:02.500000    0.002076
00:02:07.500000    0.000461
00:02:12.500000    0.000692
00:02:17.500000    0.001153
00:02:22.500000    0.000231
00:02:32.500000    0.000692
00:02:37.500000    0.000692
00:02:42.500000    0.000461
00:02:52.500000    0.000461
00:02:57.500000    0.000231
00:03:02.500000    0.001153
00:03:27.50

### 2. Conditional Density
Conditional Density 2 şekilde hesaplanır. x eksenine veya y eksenine göre(x:duration, y:amplitude temsil eder).<br/>
1. **Conditional Density / y** için: joint density'deki(**joint**) her değer, karşılık gelen dikey toplama(**vertical_total**) bölünerek hesaplanır.
2. **Conditional Density / x** için: joint density'deki(**joint**) her değer, karşılık gelen yatay toplama(**horizontal_total**) bölünerek hesaplanır.
```
Örnek Tablo (Conditional Density / y)
| xy | 4   | 6        | 8        |
|----|-----|----------|----------|
| 1  | 0,3 | 0,703125 | 0,381... |
| 2  | 0,2 | 0,1875   | 0,418..  |
| 3  | 0,5 | 0,109375 | 0,2      |
|    | 1   | 1        | 1        |
```

In [175]:
# Conditional Density / y
(joint/vertical_total)

6AU8_6BU8,-0.455000,-0.242500,-0.068050,-0.010029,0.009321,0.048000,0.067350,0.202500,0.241500,0.996000
duration,,,,,,,,,,
00:00:02.500000,NaN,NaN,1.0,0.043478,0.293623,1.0,1.0,1.0,1.0,1.0
00:00:07.500000,1.0,1.0,NaN,0.043478,0.227517,NaN,NaN,NaN,NaN,NaN
00:00:12.500000,NaN,NaN,NaN,0.065217,0.139687,NaN,NaN,NaN,NaN,NaN
00:00:17.500000,NaN,NaN,NaN,NaN,0.092502,NaN,NaN,NaN,NaN,NaN
00:00:22.500000,NaN,NaN,NaN,0.086957,0.060733,NaN,NaN,NaN,NaN,NaN
00:00:27.500000,NaN,NaN,NaN,0.152174,0.033637,NaN,NaN,NaN,NaN,NaN
00:00:32.500000,NaN,NaN,NaN,0.195652,0.031535,NaN,NaN,NaN,NaN,NaN
00:00:37.500000,NaN,NaN,NaN,0.065217,0.019855,NaN,NaN,NaN,NaN,NaN
00:00:42.500000,NaN,NaN,NaN,0.043478,0.019622,NaN,NaN,NaN,NaN,NaN


In [181]:
# Conditional Density / x
(joint.T/horizontal_total)

duration,00:00:02.500000,00:00:07.500000,00:00:12.500000,00:00:17.500000,00:00:22.500000,00:00:27.500000,00:00:32.500000,00:00:37.500000,00:00:42.500000,00:00:47.500000,...,00:02:57.500000,00:03:02.500000,00:03:27.500000,00:03:32.500000,00:03:47.500000,00:03:52.500000,00:04:17.500000,00:05:12.500000,00:05:27.500000,00:05:42.500000
6AU8_6BU8,,,,,,,,,,,,,,,,,,,,,
-0.455000,NaN,0.001022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-0.242500,NaN,0.001022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-0.068050,0.000791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-0.010029,0.001581,0.002045,0.004992,NaN,0.015152,0.046358,0.0625,0.034091,0.023256,0.017544,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.009321,0.993676,0.995910,0.995008,1.0,0.984848,0.953642,0.9375,0.965909,0.976744,0.982456,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
0.048000,0.000791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.067350,0.000791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.202500,0.000791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.241500,0.000791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
